前面幾章操作下來，我們已經成功的匯出`.pb`的模型，接下來就可以將模型佈上正式區，TensorFlow一樣提供方便的套件讓我們使用，為了方便，整個過程都是以docker來操作

[官方連結](https://www.tensorflow.org/tfx/serving/docker)

官方連結上有著詳細的說明，可以先將image接下來之後再依實際上的資料結構啟動容器，後面我們假設已經成功啟動容器。 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'2.1.0'

取得MNIST資料集並做標準化。

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train / 255., -1)
x_test = np.expand_dims(x_test / 255., -1)

In [4]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

載入需求套件

In [5]:
import json
import requests

如果是post的話，那資料格式為
```json
{
    "signature_name": "需要調用的函數簽名（Sequential模式不需要）",
    "instances": 輸入資料
}
```

而得到的json為
```json
{
    "predictions": 返回值
}
```

資料序列化

In [6]:
data = json.dumps({
    "instances": np.expand_dims(x_train[0], 0).tolist()
})

設置header

In [7]:
headers = {"content-type": "application/json"}

利用`request.post`取得回應

In [8]:
json_response = requests.post(
    'http://10.56.34.180:8501/v1/models/MLP:predict',
    data=data,
    headers=headers
)

得到200的回應，成功

In [9]:
json_response

<Response [200]>

In [15]:
json_response.json()

{'predictions': [[0.000273156445,
   0.000197476,
   0.00171972613,
   0.146828488,
   7.99330508e-07,
   0.848601043,
   0.000133246474,
   0.000922977109,
   0.000853003934,
   0.000470101426]]}

如果你跟我一樣是利用docker來執行的話，那要注意一點：
```shell
docker run --runtime=nvidia -p 8501:8501 \
  -v /home/marty/deepLearning/tf2/model:/models/MLP \
  -e MODEL_NAME=MLP \
  -t tensorflow/serving:latest-gpu &
```

那就是，在mount資料夾的時候，在container內的路徑一定是`/models/your model`，而且你並不需要指定版本，因為系統會自動去取最新版本。

下面是我們利用MNIST_11的說明所保存版本的資料結構，只要指定到saved_model_files即可
```
/saved_model_files
    /1      # 版本號為1的模型文件
        /assets
        /variables
        saved_model.pb
    ...
    /N      # 版本號為N的模型文件
        /assets
        /variables
        saved_model.pb
```

針對繼承`tf.keras.Model`的類別有幾點要注意：
1. `call`的部份一定需要以`@tf.function`裝飾
2. `@tf.function`的參數要加入指定輸入的shape
3. `tf.saved_model.save`匯出模型的時候，要利用參數`signature`來提供函數的Signature

```python
class MLP(tf.keras.Model):
    ...

    @tf.function(input_signature=[tf.TensorSpec([None, 28, 28, 1], tf.float32)])
    def call(self, inputs):
        ...
```

上面範例指出，我們在裝飾函數中指定預計輸入的維度，記得匯出模型的時候要指定signatures

```python
tf.saved_model.save(model, "saved_with_signature/1", signatures={"call": model.call})
```

在呼叫api的時候就記得指定`signature_name`：  
```python 
data = json.dumps({
    "signature_name": "call",
    "instances": data_loader.test_data[0:10].tolist()
    })
```